In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
birdnet_original_path = "/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model_FP32.tflite"
interpreter = tf.lite.Interpreter(model_path=birdnet_original_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input:", input_details)
print("Output:", output_details)

In [ ]:
interpreter = tf.lite.Interpreter(model_path=birdnet_original_path)

layer_details = interpreter.get_tensor_details()
interpreter.allocate_tensors()

for layer in layer_details:
    print("\nLayer Name: {}".format(layer['name']))
    print("\tIndex: {}".format(layer['index']))
    print("\n\tShape: {}".format(layer['shape']))

    try:
        tensor = interpreter.get_tensor(layer['index'])
        if tensor.shape == [0]:
            print("\tTensor: Ignored")
        else:
            print("\tTensor: {}".format(tensor.shape))
    except ValueError as e:
        print("\tError: {}".format(e))
        continue

    print("\tTensor Type: {}".format(tensor.dtype))
    print("\tQuantisation Parameters")
    print("\t\tScales: {}".format(layer['quantization_parameters']['scales'].shape))
    print("\t\tScales Type: {}".format(layer['quantization_parameters']['scales'].dtype))
    print("\t\tZero Points: {}".format(layer['quantization_parameters']['zero_points']))
    print("\t\tQuantized Dimension: {}".format(layer['quantization_parameters']['quantized_dimension']))


In [ ]:
tf.lite.experimental.Analyzer.analyze(model_path=birdnet_original_path,
                                      model_content=None,
                                      gpu_compatibility=False)

In [ ]:
index_class = output_details[0]["index"]
print("Outputlayer (Classification) Index:", index_class)

index_embeddings = output_details[0]["index"] -1
print("Outputlayer (Embeddings) Index:", index_embeddings)

In [ ]:
interpreter.allocate_tensors()
interpreter.invoke()

output_embeddings = interpreter.get_tensor(index_embeddings)
print("Outputlayer (Embeddings) Output:", output_embeddings)
print("Outputlayer (Embeddings) Size:", len(output_embeddings[0]))

In [ ]:
output_Classification = interpreter.get_tensor(index_class)
print("Outputlayer (Classification) Output:", output_Classification)
print("Outputlayer (Classification) Size:", len(output_Classification[0]))

### Build own classifier with two classes
---

In [ ]:
from tensorflow import keras

# Build a simple two-layer linear classifier
custom_classifier = keras.Sequential()

# Input layer with 1024 input size
custom_classifier.add(keras.layers.InputLayer(input_shape=1024))

# Hidden layer
custom_classifier.add(keras.layers.Dense(3, activation='relu'))

# Classification layer with number of classes
custom_classifier.add(keras.layers.Dense(2))

# Activation layer
custom_classifier.add(keras.layers.Activation('sigmoid'))

 **Check Shape of self build classifier:**

In [ ]:
custom_classifier.summary()

**protobuf model convert to keras model**

In [1]:
import tensorflow as tf

proto_model = tf.saved_model.load('/home/jovyan/models/origin/BirdNET-Analyzer/checkpoints/V2.3/BirdNET_GLOBAL_3K_V2.3_Model')



2023-08-08 12:57:11.373845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 12:57:11.416622: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-08 12:57:11.417136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 12:57:12.221963: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
proto_model.summary()

**pop layer with tensorflow**

In [2]:
tensor_details = interpreter.get_tensor_details()
last_layer_index = None
for index, tensor in enumerate(tensor_details):
    if tensor['index'] == 491:
        last_layer_index = index
        print("Last Layer with Index " + str(last_layer_index) + " found!")
        break

print("Before removal - tensor_details length:", len(tensor_details))
tensor_details.pop(last_layer_index)
print("After removal - tensor_details length:", len(tensor_details))

interpreter.allocate_tensors()

# Check if tensor with Index 491 is still accessible
try:
    tensor_491 = interpreter.tensor(491)
    print("Tensor with Index 491 still accessible.")
except IndexError:
    print("Tensor with Index 491 is no longer accessible.")

NameError: name 'interpreter' is not defined